In [1]:
import time
import pandas as pd
import numpy as np
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

There are many insignificant features to our work, like codes, PK, FK and out of context. 
Based on prior works and experience of especialists, will be taken off nonsense features. 
A study was carried out on the evolution and correspondence of these resources in the ENEM and Census
data for the period 2009-2018. See (featuresOrganisaton.xls)

In [2]:
teacher = ['FK_COD_DOCENTE', 'PK_COD_ENTIDADE','ID_ESPECIALIZACAO','ID_MESTRADO','ID_DOUTORADO']
school = ['ANO_CENSO','PK_COD_ENTIDADE', 'FK_COD_MUNICIPIO','FK_COD_ESTADO','ID_DEPENDENCIA_ADM','ID_AGUA_INEXISTENTE',
                  'ID_ESGOTO_INEXISTENTE','ID_ENERGIA_INEXISTENTE',
                   'ID_LABORATORIO_INFORMATICA','ID_LABORATORIO_CIENCIAS','ID_SALA_ATENDIMENTO_ESPECIAL',
                   'ID_BIBLIOTECA','ID_SALA_LEITURA','ID_SANITARIO_FORA_PREDIO',
                   'ID_SANITARIO_PNE','ID_DEPENDENCIAS_PNE','NUM_SALAS_UTILIZADAS','NUM_EQUIP_TV',
                   'NUM_EQUIP_DVD','NUM_EQUIP_COPIADORA','NUM_EQUIP_IMPRESSORA','NUM_COMP_ADMINISTRATIVOS',
                   'NUM_COMP_ALUNOS','ID_INTERNET','ID_BANDA_LARGA','NUM_FUNCIONARIOS','ID_ALIMENTACAO']

enem = ['NU_ANO','COD_ESCOLA','IN_TP_ENSINO','IDADE','TP_SEXO','TP_COR_RACA','ST_CONCLUSAO','NOTA_CN',
        'NOTA_CH','NOTA_LC','NOTA_MT','NU_NOTA_REDACAO','Q001','Q002','Q004','Q003'
]


In [3]:
#Students of ENEM
E = pd.read_csv('~/data/enem/2013/DADOS/MICRODADOS_ENEM_2013.csv', sep=';', encoding="iso-8859-2", usecols = enem)

In [4]:
#School census data
CE =    pd.read_csv("~/data/censo/2013/DADOS/ESCOLAS.CSV", sep='|', encoding="iso-8859-2", usecols = school)

In [5]:
#Teacher census data by Braazilian mesoregion
CD_CO = pd.read_csv("~/data/censo/2013/DADOS/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_NE = pd.read_csv("~/data/censo/2013/DADOS/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_N =  pd.read_csv("~/data/censo/2013/DADOS/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_SE = pd.read_csv("~/data/censo/2013/DADOS/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_S =  pd.read_csv("~/data/censo/2013/DADOS/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)

#All teacher census data
CT = pd.concat([CD_CO, CD_NE, CD_N, CD_SE, CD_S])

In [6]:
ENEM = E.copy()
C_SCHOOL = CE.copy()
C_TEACHER = CT.copy()

In [7]:
#ordering
ENEM = ENEM[enem]

newNames = [
'CO_ANO',
'CO_ESCOLA',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'TP_ST_CONCLUSAO',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL'
]

#Rename
ENEM.columns = newNames

print('checking year',ENEM.CO_ANO.unique())
print('checking key',ENEM.CO_ESCOLA.dtypes)

checking year [2013]
checking key float64


In [8]:
C_SCHOOL = C_SCHOOL[school]

newNames = ['ANO_CENSO',
'CO_ESCOLA',
 'CO_MUNICIPIO',
 'CO_UF',
 'CO_DEPENDENCIA_ADM',
 'IN_AGUA_INEXISTENTE',
 'IN_ESGOTO_INEXISTENTE',
 'IN_ENERGIA_INEXISTENTE',
 'IN_LABORATORIO_INFORMATICA',
 'IN_LABORATORIO_CIENCIAS',
 'IN_SALA_ATENDIMENTO_ESPECIAL',
 'IN_BIBLIOTECA',
 'IN_SALA_LEITURA',
 'IN_BANHEIRO_FORA_PREDIO',
 'IN_BANHEIRO_PNE',
 'IN_DEPENDENCIAS_PNE',
 'QT_SALAS_UTILIZADAS',
 'QT_EQUIP_TV',
 'QT_EQUIP_DVD',
 'QT_EQUIP_COPIADORA',
 'QT_EQUIP_IMPRESSORA',
 'QT_COM_ADMINISTRATIVO',
 'QT_COMP_ALUNO',
 'IN_INTERNET',
 'IN_BANDA_LARGA',
 'QT_FUNCIONARIOS',
 'IN_ALIMENTACAO'
]

#Rename 
C_SCHOOL.columns =  newNames

print('Checking year of censo',C_SCHOOL.ANO_CENSO.unique())
C_SCHOOL.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_SCHOOL.CO_ESCOLA.dtypes)

Checking year of censo [2013]
checking key int64


In [9]:
C_TEACHER = C_TEACHER[teacher]

# New default column names for all Datasets
newNames = [
'CO_PROFESSOR',
'CO_ESCOLA',  
'IN_ESPECIALIZACAO',
'IN_MESTRADO',
'IN_DOUTORADO'
]
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
C_TEACHER.columns = newNames
C_TEACHER.drop_duplicates(subset=['CO_PROFESSOR', 'CO_ESCOLA'], inplace=True)
C_TEACHER.drop('CO_PROFESSOR', axis=1, inplace = True)

print('checking key',C_TEACHER.CO_ESCOLA.dtypes)

checking key int64


### Minimum Scope Definition

In [10]:
ENEM.dropna(inplace = True, axis=0, subset = ['CO_ESCOLA'])

In [11]:
#should care about the nulls into scope filter features?
filters = (['TP_ST_CONCLUSAO', 'TP_ENSINO'])
ENEM[filters].isnull().sum()
#NO, IT'S LOWER


TP_ST_CONCLUSAO       0
TP_ENSINO          7297
dtype: int64

In [12]:
print('drop out',((ENEM.TP_ENSINO!= 1)|(ENEM.TP_ENSINO!= 3)).sum(), 'who not attend regular and prof approachs schools')
ENEM = ENEM.loc[((ENEM.TP_ENSINO== 1)|(ENEM.TP_ENSINO== 3))]
ENEM.drop('TP_ENSINO', inplace = True, axis =1)

drop out 1635848 who not attend regular and prof approachs schools


In [13]:
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
for i in fields:
    print ('null:', i, ENEM[i].isnull().sum())
    print ('zero:', i, (ENEM[i]== 0).sum())
    
    #, "and", ENEM.I==0)


null: NU_NOTA_CN 180365
zero: NU_NOTA_CN 0
null: NU_NOTA_CH 180365
zero: NU_NOTA_CH 0
null: NU_NOTA_LC 202699
zero: NU_NOTA_LC 0
null: NU_NOTA_MT 202699
zero: NU_NOTA_MT 0
null: NU_NOTA_REDACAO 0
zero: NU_NOTA_REDACAO 223663


In [14]:
ENEM.dropna(inplace = True, axis=0, subset=fields)
ENEM = ENEM.loc[~(ENEM[fields] == 0).any(axis=1)]

In [15]:
ENEM[fields].dtypes

NU_NOTA_CN         float64
NU_NOTA_CH         float64
NU_NOTA_LC         float64
NU_NOTA_MT         float64
NU_NOTA_REDACAO      int64
dtype: object

In [16]:
#ONLY STUDENTS AT  17 -19 YEAR OLD. WE WANT TO GET ONLY REGULAR STUDENTS THAT ARE IN LAST YEAR OF SECONDARY SCHOOL. 
#THE MICRODADOS HAVE A FIELD TO CONTROL IT. BUT IT IS NOT CONFIDENCE,DUE A TURN OUT TEAACHERS THAT EXTEND ACADEMIC YEAR
print('null', ENEM.NU_IDADE.isnull().sum())
#firts, fill NA with the mean of student school
ENEM.NU_IDADE = ENEM.groupby(ENEM['CO_ESCOLA'])['NU_IDADE'].apply(lambda x: x.fillna(round(x.mean())))
ENEM.NU_IDADE = ENEM.NU_IDADE.astype('int32')
print('taking off',((ENEM.NU_IDADE <17)|(ENEM.NU_IDADE>19)).sum())
ENEM = ENEM.loc[(ENEM.NU_IDADE >16) &  (ENEM.NU_IDADE<20)]
ENEM.NU_IDADE.value_counts()

null 71
taking off 137955


17    636953
18    348404
19    101089
Name: NU_IDADE, dtype: int64

In [17]:
ENEM[fields].describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,1.086446e+06,1.086446e+06,1.086446e+06,1.086446e+06,1.086446e+06
mean,4.788045e+02,5.206266e+02,4.955303e+02,5.230591e+02,5.326338e+02
std,7.692836e+01,8.328508e+01,7.643319e+01,1.045678e+02,1.410130e+02
min,3.341000e+02,2.995000e+02,2.865000e+02,3.273000e+02,3.400000e+01
25%,4.206000e+02,4.625000e+02,4.421000e+02,4.447000e+02,4.400000e+02
50%,4.675000e+02,5.166000e+02,4.983000e+02,5.165000e+02,5.200000e+02
75%,5.281000e+02,5.755000e+02,5.494000e+02,5.907000e+02,6.200000e+02
max,8.871000e+02,8.855000e+02,8.133000e+02,9.715000e+02,1.000000e+03


### Transform features to default categories

In [18]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL']
ENEM[fields].isnull().sum()


TP_SEXO            0
TP_COR_RACA        0
EDU_PAI            0
EDU_MAE            0
QT_PESSOAS_CASA    0
RENDA_MENSAL       0
dtype: int64

In [19]:
for i in fields:
    print(i,ENEM[i].value_counts().index)

TP_SEXO Index(['F', 'M'], dtype='object')
TP_COR_RACA Int64Index([1, 3, 2, 4, 0, 5], dtype='int64')
EDU_PAI Index(['E', 'B', 'C', 'G', 'I', 'D', 'H', 'F', 'A'], dtype='object')
EDU_MAE Index(['E', 'C', 'B', 'G', 'D', 'H', 'F', 'I', 'A'], dtype='object')
QT_PESSOAS_CASA Int64Index([4, 3, 5, 6, 2, 7, 8, 9, 1, 10, 11, 12, 13, 14, 15, 20, 16, 17, 19,
            18],
           dtype='int64')
RENDA_MENSAL Index(['B', 'C', 'D', 'F', 'E', 'G', 'H', 'I', 'K', 'J', 'Q', 'N', 'A', 'L',
       'O', 'M', 'P'],
      dtype='object')


In [20]:
#Order relation, is better for numeric categories.
categorical = ['TP_SEXO','EDU_PAI', 'EDU_MAE', 'RENDA_MENSAL']
for i in categorical:
    codes, uniques = pd.factorize(ENEM[i], sort=True)
    ENEM[i] = codes

In [21]:
ENEM['EDU_PAI_TEMP'] = ENEM['EDU_PAI']
ENEM['EDU_MAE_TEMP'] = ENEM['EDU_MAE']
ENEM['QT_PESSOAS_CASA_TEMP'] = ENEM['QT_PESSOAS_CASA']
ENEM['RENDA_MENSAL_TEMP'] = ENEM['RENDA_MENSAL']


In [22]:
# Paring with default categories

ENEM.loc[ENEM['EDU_PAI']==0, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==1, 'EDU_PAI_TEMP']=1
ENEM.loc[ENEM['EDU_PAI']==2, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==3, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==4, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==5, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==6, 'EDU_PAI_TEMP']=4
ENEM.loc[ENEM['EDU_PAI']==7, 'EDU_PAI_TEMP']=5
ENEM.loc[ENEM['EDU_PAI']==8, 'EDU_PAI_TEMP']=0


ENEM.loc[ENEM['EDU_MAE']==0, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==1, 'EDU_MAE_TEMP']=1
ENEM.loc[ENEM['EDU_MAE']==2, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==3, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==4, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==5, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==6, 'EDU_MAE_TEMP']=4
ENEM.loc[ENEM['EDU_MAE']==7, 'EDU_MAE_TEMP']=5
ENEM.loc[ENEM['EDU_MAE']==8, 'EDU_MAE_TEMP']=0


ENEM.loc[ENEM['QT_PESSOAS_CASA']==1, 'QT_PESSOAS_CASA_TEMP']=0
ENEM.loc[ENEM['QT_PESSOAS_CASA']==2, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==3, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==4, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==5, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==6, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==7, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']>7, 'QT_PESSOAS_CASA_TEMP']=3

ENEM.loc[ENEM['RENDA_MENSAL']==0, 'RENDA_MENSAL_TEMP']=0
ENEM.loc[ENEM['RENDA_MENSAL']==1, 'RENDA_MENSAL_TEMP']=1
ENEM.loc[ENEM['RENDA_MENSAL']==2, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==3, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[(ENEM['RENDA_MENSAL']>3) & (ENEM['RENDA_MENSAL']<8), 'RENDA_MENSAL_TEMP']=3
ENEM.loc[(ENEM['RENDA_MENSAL']>7) & (ENEM['RENDA_MENSAL']<13), 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']>12, 'RENDA_MENSAL_TEMP']=5



In [23]:
#back to original columns
ENEM['EDU_PAI'] = ENEM['EDU_PAI_TEMP']
ENEM['EDU_MAE'] = ENEM['EDU_MAE_TEMP']
ENEM['QT_PESSOAS_CASA'] = ENEM['QT_PESSOAS_CASA_TEMP']
ENEM['RENDA_MENSAL'] = ENEM['RENDA_MENSAL_TEMP']



In [24]:
#drop temps

ENEM.drop(['EDU_PAI_TEMP', 'EDU_MAE_TEMP', 'QT_PESSOAS_CASA_TEMP','RENDA_MENSAL_TEMP'], axis=1, inplace=True)

In [25]:
print('Total Students at ENEM:', ENEM.shape[0], 'of', ENEM.drop_duplicates('CO_ESCOLA').shape[0], 'schools')
print('Total Schools at Census:', C_SCHOOL.shape[0])

            

#######################################
print('########### Only about Schools at ENEM:')

schoolsEnem = ENEM['CO_ESCOLA'].dropna().unique()
schoolsCenso = C_SCHOOL['CO_ESCOLA'].dropna().unique()
schoolsTeacher = C_TEACHER['CO_ESCOLA'].dropna().unique()
schoolsEnem2 = ENEM['CO_ESCOLA'].dropna()

enem_censo = np.setdiff1d(schoolsEnem ,schoolsCenso)
print('Schools at ENEM out of School Census', sum(np.isin(schoolsEnem, schoolsCenso, invert=True)))

print('Schools at ENEM out of Teacher Census', sum(np.isin(schoolsEnem, schoolsTeacher, invert=True)))
print('Schools at ENEM and School Census out of Teacher Census', sum(np.isin(enem_censo, schoolsTeacher)))

print('Students Wasted*****', sum(np.isin(schoolsEnem2, schoolsCenso, invert=True)))

Total Students at ENEM: 1086446 of 27758 schools
Total Schools at Census: 268166
########### Only about Schools at ENEM:
Schools at ENEM out of School Census 160
Schools at ENEM out of Teacher Census 624
Schools at ENEM and School Census out of Teacher Census 0
Students Wasted***** 1604


### New Features

ENEM

In [26]:
#Final Media, Target
ENEM['NU_NOTA_GERAL'] = (ENEM.NU_NOTA_CN + ENEM.NU_NOTA_CH + ENEM.NU_NOTA_LC + ENEM.NU_NOTA_MT + ENEM.NU_NOTA_REDACAO)/5

SCHOOL CENSUS - Itś better include after merge all datasets years.


TEACHER CENSUS

In [27]:
C_TEACHER.isnull().sum()

CO_ESCOLA            0
IN_ESPECIALIZACAO    0
IN_MESTRADO          0
IN_DOUTORADO         0
dtype: int64

In [28]:
print('number of teacher who work at an school listed at ENEM and do not have study level assigned:',
      C_TEACHER['CO_ESCOLA'].isin(ENEM['CO_ESCOLA']).sum())


#lets fill with lower level

C_TEACHER['IN_ESPECIALIZACAO'].fillna(0, inplace = True)
C_TEACHER['IN_MESTRADO'].fillna(0, inplace = True)
C_TEACHER['IN_DOUTORADO'].fillna(0, inplace = True)

number of teacher who work at an school listed at ENEM and do not have study level assigned: 1003958


In [29]:
C_TEACHER['TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 1

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_DOUTORADO ==1.0), 'TITULACAO'] = 3

#Rebuild level education information as indicator of unique level.
C_TEACHER['NU_GRADUACAO'] = 0
C_TEACHER['NU_ESPECIALIZACAO'] = 0
C_TEACHER['NU_MESTRADO'] = 0
C_TEACHER['NU_DOUTORADO'] = 0

C_TEACHER.loc[C_TEACHER.TITULACAO == 0, 'NU_GRADUACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 1, 'NU_ESPECIALIZACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 2, 'NU_MESTRADO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 3, 'NU_DOUTORADO']= 1


C_TEACHER['NU_GRADUACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_GRADUACAO'].transform(sum)
C_TEACHER['NU_ESPECIALIZACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_ESPECIALIZACAO'].transform(sum)
C_TEACHER['NU_MESTRADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_MESTRADO'].transform(sum)
C_TEACHER['NU_DOUTORADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_DOUTORADO'].transform(sum)



#Calculating weighted indice of teacher education
SUM= C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform(sum)
COUNT =  C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform('count')
C_TEACHER['TITULACAO'] = (SUM)/(COUNT*3)  

##arranjar isso dai



#transformin to school grain
C_TEACHER.drop_duplicates('CO_ESCOLA', inplace = True)

## STUDENT GRAIN 

In [30]:
# teacher to Schoool Grain
print('C_SCHOOL',C_SCHOOL.shape)
print('C_TEACHER',C_TEACHER.shape)
CENSO19 = pd.merge(C_SCHOOL, C_TEACHER, on = 'CO_ESCOLA', how = 'inner')
print('C_SCHOOL + C_TEACHER',CENSO19.shape)

#Now, all to Student Grain
print('ENEM',ENEM.shape)
ENEM_CENSO_19 = pd.merge(ENEM, CENSO19, on = 'CO_ESCOLA', how = 'inner')
print('FINAL',ENEM_CENSO_19.shape)

ENEM_CENSO_19.to_csv('STUDENT.csv')

C_SCHOOL (268166, 26)
C_TEACHER (192678, 9)
C_SCHOOL + C_TEACHER (192678, 34)
ENEM (1086446, 16)
FINAL (1082766, 49)


In [31]:
ENEM_CENSO_19.head()

,CO_ANO,CO_ESCOLA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,NU_GRADUACAO,NU_ESPECIALIZACAO,NU_MESTRADO,NU_DOUTORADO
0,2013,41001001.0,19,0,1,2,360.8,475.5,379.4,502.7,380,2,2,2,1,419.68,4111308,41,2,0.0,0.0,0.0,1,1,0,1,0,1,0,0,3.0,5,3,0,5,8.0,27.0,1.0,0.0,26.0,1.0,1,0,0,0.288889,2,13,0,0
1,2013,41001001.0,17,0,3,2,417.0,499.6,490.4,515.6,400,2,2,1,2,464.52,4111308,41,2,0.0,0.0,0.0,1,1,0,1,0,1,0,0,3.0,5,3,0,5,8.0,27.0,1.0,0.0,26.0,1.0,1,0,0,0.288889,2,13,0,0
2,2013,41001001.0,19,0,3,2,440.4,494.9,504.9,501.5,600,2,0,1,1,508.34,4111308,41,2,0.0,0.0,0.0,1,1,0,1,0,1,0,0,3.0,5,3,0,5,8.0,27.0,1.0,0.0,26.0,1.0,1,0,0,0.288889,2,13,0,0
3,2013,41001001.0,17,0,1,2,344.4,423.8,362.0,405.1,540,3,5,1,3,415.06,4111308,41,2,0.0,0.0,0.0,1,1,0,1,0,1,0,0,3.0,5,3,0,5,8.0,27.0,1.0,0.0,26.0,1.0,1,0,0,0.288889,2,13,0,0
4,2013,41001001.0,17,0,1,2,373.9,377.2,338.1,331.9,560,2,2,2,3,396.22,4111308,41,2,0.0,0.0,0.0,1,1,0,1,0,1,0,0,3.0,5,3,0,5,8.0,27.0,1.0,0.0,26.0,1.0,1,0,0,0.288889,2,13,0,0


## SCHOOL GRAIN

In [32]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL', 'NU_IDADE', 'NU_NOTA_CN',
             'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','NU_NOTA_GERAL' ]

for i in fields:   
    
            ENEM[i] = ENEM.groupby('CO_ESCOLA')[i].transform('mean')
            

In [33]:
# SCHOOL GRAIN AND SHOW THE NEW NOTA DISTRIBUTION
ENEM.drop_duplicates('CO_ESCOLA', inplace=True)
#print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

In [34]:
#recalculating target for new  Nota Geral distribution 
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

74.99819871748686 % lowers quartis


In [35]:
print('censo before',CENSO19.shape)
print('enem_school before', ENEM.shape)
CENSO_ENEM_19 = pd.merge(CENSO19, ENEM, on='CO_ESCOLA', how='inner')
print(CENSO_ENEM_19.shape)

CENSO_ENEM_19.to_csv('SCHOOL.csv', index=False)

censo before (192678, 34)
enem_school before (27758, 17)
(27134, 50)


In [36]:
CENSO_ENEM_19.head()

,CO_ESCOLA,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,NU_GRADUACAO,NU_ESPECIALIZACAO,NU_MESTRADO,NU_DOUTORADO,CO_ANO,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,TARGET
0,35100894,3550308,35,4,0.0,0.0,0.0,1,0,0,1,1,1,0,0,8.0,3,2,1,5,5.0,20.0,1.0,1.0,40.0,0.0,1,0,0,0.055556,15,3,0,0,2013,19.000000,1.000000,2.000000,2,452.550000,556.200000,501.800000,555.250000,480.000000,3.000000,3.000000,1.500000,3.500000,509.160000,0
1,35030521,3511904,35,2,0.0,0.0,0.0,1,0,0,0,1,0,0,0,10.0,4,2,1,5,8.0,7.0,1.0,1.0,45.0,1.0,0,0,0,0.045977,25,4,0,0,2013,17.125000,0.500000,1.343750,2,448.640625,501.293750,468.553125,478.928125,498.750000,2.406250,2.687500,1.718750,2.625000,479.233125,0
2,35030661,3516903,35,2,0.0,0.0,0.0,1,1,0,0,1,0,0,1,12.0,6,2,2,7,8.0,8.0,1.0,1.0,47.0,1.0,0,0,0,0.101449,16,7,0,0,2013,17.145161,0.403226,1.419355,2,465.270968,509.887097,482.482258,517.446774,515.483871,2.016129,2.500000,1.677419,2.129032,498.114194,0
3,35019239,3513900,35,2,0.0,0.0,0.0,1,0,0,1,0,0,0,1,12.0,3,1,1,7,6.0,11.0,1.0,1.0,70.0,1.0,1,0,0,0.118280,20,11,0,0,2013,17.258621,0.344828,1.172414,2,453.991379,494.305172,473.544828,498.955172,529.310345,1.689655,2.137931,1.775862,2.120690,490.021379,0
4,35016329,3552205,35,2,0.0,0.0,0.0,1,0,0,0,1,0,1,0,17.0,5,2,1,8,9.0,17.0,0.0,NaN,109.0,1.0,1,0,0,0.111111,43,19,1,0,2013,17.435294,0.411765,1.741176,2,454.874118,498.949412,484.415294,493.328235,471.058824,2.164706,2.552941,1.694118,2.470588,480.525176,0


In [37]:
CENSO_ENEM_19.isnull().sum()

CO_ESCOLA                          0
CO_MUNICIPIO                       0
CO_UF                              0
CO_DEPENDENCIA_ADM                 0
IN_AGUA_INEXISTENTE                0
IN_ESGOTO_INEXISTENTE              0
IN_ENERGIA_INEXISTENTE             0
IN_LABORATORIO_INFORMATICA         0
IN_LABORATORIO_CIENCIAS            0
IN_SALA_ATENDIMENTO_ESPECIAL       0
IN_BIBLIOTECA                      0
IN_SALA_LEITURA                    0
IN_BANHEIRO_FORA_PREDIO            0
IN_BANHEIRO_PNE                    0
IN_DEPENDENCIAS_PNE                0
QT_SALAS_UTILIZADAS                0
QT_EQUIP_TV                        0
QT_EQUIP_DVD                       0
QT_EQUIP_COPIADORA                 0
QT_EQUIP_IMPRESSORA                0
QT_COM_ADMINISTRATIVO            463
QT_COMP_ALUNO                   1769
IN_INTERNET                      267
IN_BANDA_LARGA                  1214
QT_FUNCIONARIOS                    0
IN_ALIMENTACAO                     0
IN_ESPECIALIZACAO                  0
I